In [1]:
from google.colab import drive
from os.path import join
import os

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/Colab Notebooks/'       # path to your project on Drive



drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)


Mounted at /content/drive


In [ ]:
GIT_USERNAME = "ahhuang007" # replace with yours
GIT_TOKEN = os.environ['git_key_wsb']           # definitely replace with yours
GIT_REPOSITORY = "wallstreetbets"      # ...nah


In [16]:
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already   
%cd "{PROJECT_PATH}"

mkdir: cannot create directory ‘/content/drive/My Drive/Colab Notebooks/wallstreetbets’: File exists
/content/drive/My Drive/Colab Notebooks/wallstreetbets


In [32]:
#GIT_PATH = "https://GIT_TOKEN@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
!git clone "https://$git_key_wsb@github.com/ahhuang007/wallstreetbets.git"
#!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./

Cloning into 'wallstreetbets'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (362/362), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 362 (delta 202), reused 293 (delta 133), pack-reused 0
Receiving objects: 100% (362/362), 3.16 MiB | 10.17 MiB/s, done.
Resolving deltas: 100% (202/202), done.


In [33]:
%cd wallstreetbets

/content/drive/MyDrive/Colab Notebooks/wallstreetbets


In [19]:
!mv ./wallstreetbets/* "{PROJECT_PATH}"

In [31]:
!rm -rf ./wallstreetbets

Pulling data from Github

In [ ]:
!git pull origin

Adding stuff to Github

In [57]:
!git add .

In [58]:
!git config --global user.email "ahhuang007@gmail.com"
!git config --global user.name "ahhuang007"

In [59]:
!git commit -m "tinkered with dataloader, made basic NN that is now compatible with dataloaders, will debug training next"

[main e69862a] tinkered with dataloader, made basic NN that is now compatible with dataloaders, will debug training next
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite colab_wsb.ipynb (94%)


In [43]:
!git remote set-url origin https://$git_key_wsb@github.com/ahhuang007/wallstreetbets.git

In [60]:
!git push origin

Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 6.68 KiB | 1.67 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/ahhuang007/wallstreetbets.git
   05f68fe..e69862a  main -> main


In [45]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   colab_wsb.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


Actual Code Below

The reason for this Colab file is to explore the viability of using a NN instead of an RL policy - in theory, we already know a basic optimal policy (buy before it increases, sell before it decreases), we just need to figure out if it will increase/decrease beforehand, which a NN can maybe do.

In [2]:
import pandas as pd

In [3]:
%cd drive/MyDrive/Colab Notebooks/wallstreetbets

/content/drive/MyDrive/Colab Notebooks/wallstreetbets


In [34]:
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np

class wsb_dataset(Dataset):
  """class for formatting training data"""

  def __init__(self, df, labels):
    self.df = df
    self.labels = labels
  
  def __len__(self):
    return len(self.df)
  
  def __getitem__(self, idx):
    sample = self.df.iloc[idx, :]
    sample = np.array([sample])
    sample = sample.astype('float')#.reshape(-1, 2)
    label = self.labels[idx]
    return sample, label

In [35]:
#Gathering data, formatting it for NN
dfs = []
training_datas = []
cryptos = ['AAVE', 'ADA', 'ALGO', 'ATOM', 
           'AVAX', 'BCH', 'BTC', 'DOT', 
           'ETH', 'LINK', 'LRC', 'LTC', 
           'MANA', 'MATIC', 'SOL', 'UNI']
cryptos = ['BTC']
for c in cryptos:
    df = pd.read_csv('./data/' + c + '_data.csv')
    '''I think we'll do a similar plan to my source idea.
    5 months for training, 2 months for validation/tuning, 5 months for testing
    '''
    #dfs.append(df[38:175200].reset_index(drop = True))
    #Splitting data into training/validation/test
    labels = list(df['close'])
    #labels.append(0)
    labels = labels[1:]
    #df['label'] = labels
    df = df[:-1]
    training = wsb_dataset(df[38:175200].reset_index(drop = True), labels[38:175200])
    validation = wsb_dataset(df[175200:282600].reset_index(drop = True), labels[175200:282600])
    testing = wsb_dataset(df[262800:].reset_index(drop = True), labels[262800:])
    training_datas.append([training, validation, testing])

In [36]:
train_dataloader = DataLoader(training_datas[0][0], batch_size = 64)
valid_dataloader = DataLoader(training_datas[0][1], batch_size = 64)

In [69]:
for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(X.float())
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 10])
tensor([[[ 3.8000e+01,  1.6095e+09,  2.9150e+04,  2.9192e+04,  2.9150e+04,
           2.9188e+04,  6.1279e+05,  7.4545e-01,  7.6215e+01,  2.5390e+01]],

        [[ 3.9000e+01,  1.6095e+09,  2.9188e+04,  2.9206e+04,  2.9188e+04,
           2.9193e+04,  9.2300e+05,  3.6839e+00,  1.2652e+02,  2.1321e+01]],

        [[ 4.0000e+01,  1.6095e+09,  2.9193e+04,  2.9193e+04,  2.9171e+04,
           2.9179e+04,  7.5944e+05,  4.8805e+00,  6.8148e+01,  1.6128e+01]],

        [[ 4.1000e+01,  1.6095e+09,  2.9179e+04,  2.9183e+04,  2.9150e+04,
           2.9175e+04,  6.5547e+05,  5.4361e+00,  2.6576e+01,  1.3461e+01]],

        [[ 4.2000e+01,  1.6095e+09,  2.9175e+04,  2.9202e+04,  2.9172e+04,
           2.9190e+04,  1.0236e+06,  6.9326e+00,  7.8859e+01,  1.1481e+01]],

        [[ 4.3000e+01,  1.6095e+09,  2.9190e+04,  2.9191e+04,  2.9178e+04,
           2.9185e+04,  3.3757e+05,  7.6502e+00,  6.1423e+01,  1.1224e+01]],

        [[ 4.4000e+01,  1.6095e+0

In [79]:
##Create NN
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(10, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        #print(x)
        logits = self.linear_relu_stack(x)
        return logits


In [80]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=10, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [50]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [83]:
##Train NN

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X.float())
        loss = loss_fn(pred, y.float())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            #print(loss)
            loss, current = loss.item(), batch * len(X)
            
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [81]:
for X, y in train_dataloader:
  print(X[0])
  print(model(X[0].float()))
  break

tensor([[3.8000e+01, 1.6095e+09, 2.9150e+04, 2.9192e+04, 2.9150e+04, 2.9188e+04,
         6.1279e+05, 7.4545e-01, 7.6215e+01, 2.5390e+01]], dtype=torch.float64)
tensor([[-62328216.]], grad_fn=<AddmmBackward0>)


In [84]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X.float())
            test_loss += loss_fn(pred, y.float()).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [85]:
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(valid_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 3888475216543744.000000  [    0/175162]


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 3885518668431360.000000  [ 6400/175162]
loss: 3890299503902720.000000  [12800/175162]
loss: 3875298256879616.000000  [19200/175162]
loss: 3887220817657856.000000  [25600/175162]
loss: 3893803928780800.000000  [32000/175162]
loss: 3895999193939968.000000  [38400/175162]
loss: 3894072095801344.000000  [44800/175162]
loss: 3901723848474624.000000  [51200/175162]
loss: 3901093830459392.000000  [57600/175162]
loss: 3906616353095680.000000  [64000/175162]
loss: 3907609027411968.000000  [70400/175162]
loss: 3892697706266624.000000  [76800/175162]
loss: 3910241842364416.000000  [83200/175162]
loss: 3910289355440128.000000  [89600/175162]
loss: 3908236897943552.000000  [96000/175162]
loss: 3879674560118784.000000  [102400/175162]
loss: 3913591379984384.000000  [108800/175162]
loss: 3922231512006656.000000  [115200/175162]
loss: 3922168161239040.000000  [121600/175162]
loss: 3925417035563008.000000  [128000/175162]
loss: 3923844272226304.000000  [134400/175162]
loss: 3930492344729600.00000

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([58])) that is different to the input size (torch.Size([58, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Error: 
 Accuracy: 0.0%, Avg loss: 3948666917403686.500000 

Done!


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
##Test NN in validation

In [ ]:
##Plot/record accuracy

In [ ]:
#Downloading file
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')